In [ ]:
import os
import re
from dotenv import load_dotenv
from langgraph.graph import StateGraph
from openai import OpenAI

# Load .env variables
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ------------------------
# Custom Math Functions
# ------------------------

def plus(a, b):
    return a + b

def subtract(a, b):
    return a - b

def multiply(a, b):
    return a * b

def divide(a, b):
    if b == 0:
        return "Error: Division by zero"
    return a / b

# ------------------------
# Classifier
# ------------------------

def classify_query(state):
    text = state["content"].lower()
    if any(op in text for op in ["plus", "add", "sum", "subtract", "minus", "multiply", "times", "divide", "by"]):
        return "math"
    return "general"

# ------------------------
# Extractor
# ------------------------

def extract_operator_and_numbers(text):
    numbers = list(map(float, re.findall(r"[-+]?\d*\.\d+|\d+", text)))
    text = text.lower()
    if "plus" in text or "add" in text or "sum" in text:
        return plus, numbers
    elif "subtract" in text or "minus" in text:
        return subtract, numbers
    elif "multiply" in text or "times" in text:
        return multiply, numbers
    elif "divide" in text or "by" in text:
        return divide, numbers
    return None, numbers

# ------------------------
# Nodes
# ------------------------

def math_node(state):
    query = state["content"]
    func, nums = extract_operator_and_numbers(query)
    if func and len(nums) >= 2:
        result = func(nums[0], nums[1])
        return {"content": f"The result is: {result}"}
    return {"content": "Sorry, I couldn't understand the math query."}

def llm_node(state):
    query = state["content"]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": query}]
    )
    return {"content": response.choices[0].message.content}

# ------------------------
# LangGraph Definition
# ------------------------

builder = StateGraph()

builder.add_node("math", math_node)
builder.add_node("general", llm_node)
builder.set_entry_point("router")

def router(state):
    return classify_query(state)

builder.add_conditional_edges("router", router, {
    "math": "math",
    "general": "general"
})

graph = builder.compile()

# ------------------------
# Runner
# ------------------------

if __name__ == "__main__":
    print("🤖 Math + General Agent Ready! Type 'exit' to quit.")
    while True:
        user_input = input("You: ")
        if user_input.strip().lower() in ["exit", "quit"]:
            break
        response = graph.invoke({"content": user_input})
        print("Agent:", response["content"])
